### Data Cleaning

### Steps: 
1. depot_delivered_qty_on_day remove where negative
2. closing_inventory_on_day remove where negative
3. waste_value_on_day remove where negative
4. Remove rows where (instore['stock_out_ind_on_day'] + instore['ranging_indicator_on_day']) != 1

In [1]:
import pandas as pd

In [2]:
fore = pd.read_csv('../data/Case Study - Forecast Data.txt', sep='\t', encoding='utf-16',
                  parse_dates=['calendar_date'])

In [3]:
depot = pd.read_csv('../data/Case Study - Depot Data.txt', sep='\t', encoding='utf-16',
                   parse_dates=['calendar_date'])

In [4]:
instore = pd.read_csv('../data/Case Study - In Store Data.txt', sep='\t', encoding='utf-16',
                     parse_dates=['calendar_date'])

In [5]:
cinv = pd.read_csv('../data/Case Study - Closing Inventory.txt', sep='\t', encoding='utf-16',
                   parse_dates=['calendar_date'])

#### 1. depot_delivered_qty_on_day remove where negative

In [6]:
depot_c = depot[~(depot['depot_delivered_qty_on_day'] < 0)]

#### 2. closing_inventory_on_day remove where negative


In [8]:
instore_c = instore[~(instore['closing_inventory_on_day'] < 0)]

#### 3. waste_value_on_day remove where negative

In [9]:
instore_c = instore_c[~(instore_c['waste_value_on_day'] < 0)]

#### 4. Remove rows where (instore['stock_out_ind_on_day'] + instore['ranging_indicator_on_day']) != 1

In [19]:
(instore['stock_out_ind_on_day'] + instore['ranging_indicator_on_day']).isna().value_counts()

False    1792461
True      856078
dtype: int64

In [20]:
# Use instore_c
instore_c = instore_c[(instore_c['stock_out_ind_on_day'] + instore_c['ranging_indicator_on_day'] == 1)]

#### 5. **Added:** Remove rows with too much missing data: stock_out_ind_on_day, ranging_indicator_on_day, closing_inventory_on_day

## Join data

**Note:** No need to use `Closing Inventory`

**Check mutual columns before joining to ensure clean joins**

In [23]:
# Are the consistent columns the same in depot and instore
mut_cols = list(depot.columns[depot.columns.isin(instore_c.columns)])
(depot[mut_cols] == instore[mut_cols]).apply(lambda x: x.value_counts()).transpose()
# or all(depot[mut_cols] == instore[mut_cols])
# yes

,True
upc,2648539
calendar_date,2648539
calendar_id,2648539
store_id,2648539
geography_id,2648539
shelf_life,2648539
units_per_tray,2648539
